In [198]:
import os
import ast
import json
import importlib 
import itertools
import collections
import pprint

import pandas as pd

from climatools.lblnew import (setup_bestfit, 
                               setup_overlap,
                               pipeline,
                               bestfit_params)

import climatools.cliradlw.setup as setup_clirad
import climatools.cliradlw.pipeline as pipeline_clirad



from IPython import display

importlib.reload(bestfit_params)
importlib.reload(setup_bestfit)
importlib.reload(setup_overlap)
importlib.reload(pipeline)
importlib.reload(bestfit_params)
importlib.reload(setup_clirad)
importlib.reload(pipeline_clirad)

<module 'climatools.cliradlw.pipeline' from '/chia_cluster/home/jackyu/climatools/climatools/cliradlw/pipeline.py'>

In [129]:
pd.set_option('max_colwidth', -1)

In [130]:
def bandmap_lblnew2clirad():
    '''
    Band map: lblnew ---> clirad
    '''
    d = {'1': '1',
                     '2': '2',
                     '3a': '3',
                     '3b': '4',
                     '3c': '5',
                     '4': '6', 
                     '5': '7',
                     '6': '8',
                     '7': '9', 
                     '8': '10',
                     '9': '11'}
    return d



def bandmap_clirad2lblnew():
    '''
    Band map: clirad ---> lblnew
    '''
    d_lblnew2clirad = bandmap_lblnew2clirad()
    return {v: k for k, v in d_lblnew2clirad.items()}



In [131]:
def get_hrefs_clirad(param=None):
    atmpros = ('mls', 'saw', 'trp')
        
    hrefs = []
    for atmpro in atmpros:
        p = param.copy()
        p['atmpro'] = atmpro
        url = pipeline_clirad.nbviewer_url(param=p, setup=setup_clirad)
        
        s = '''<a href="{}">{}</a>'''
        s = s.format(url, atmpro)
        hrefs.append(s)
    s_hrefs = ' '.join(hrefs)
    return s_hrefs



def tb_hrefs_clirad():
    df = pd.DataFrame()
    
    for param in setup_clirad.test_cases():
        
        hrefs = get_hrefs_clirad(param=param)
        
        bands = [str(b) for b in param['band']]
        
        df.loc['+'.join(param['molecule']), 
               '+'.join(bands)] = hrefs
        
    df = df.fillna('')
    
    bmap = bandmap_clirad2lblnew()
    lblnew_bands = [bmap[b] for b in df.columns]
    columns = pd.MultiIndex.from_arrays([lblnew_bands, df.columns],
                                        names=['lblnew', 'clirad'])
    df.columns = columns
    

    return df
        


In [204]:
def molecules_byband_atm():
    return {1: {'h2o': 'atmpro'},
            2: {'h2o': 'atmpro'}, 
            3: {'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07}}


def clirad_params_atm():
    d = {}
    for band, molecule in molecules_byband_atm().items():
        for param in setup_clirad.test_cases():
            if [band] == param['band'] and molecule == param['molecule']:
                d[band] = param
                break                
    return d


def analysis_dirs_atm():
    params = clirad_params_atm()
    return {band: pipeline_clirad.get_analysis_dir(param=param,
                                                   setup=setup_clirad) 
            for band, param in params.items()}


def lblnew_params_atm():
    dirs = analysis_dirs_inpractice()
    
    d = {}
    for band, dirname in dirs.items():
        with open(os.path.join(dirname, 'param.py'), 
                  mode='r', encoding='utf-8') as f:
            _, l = f.readlines()
        
        s = l.split('=')[1].strip()
        d[band] = ast.literal_eval(s)
        
    return d

    
    
pprint.pprint(clirad_params_atm())

{1: {'atmpro': 'mls',
     'band': [1],
     'commitnumber': 'de5d43e',
     'molecule': {'h2o': 'atmpro'},
     'tsfc': 294},
 2: {'atmpro': 'mls',
     'band': [2],
     'commitnumber': 'de5d43e',
     'molecule': {'h2o': 'atmpro'},
     'tsfc': 294},
 3: {'atmpro': 'mls',
     'band': [3],
     'commitnumber': 'de5d43e',
     'molecule': {'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07},
     'tsfc': 294}}


In [133]:
def script():
    df = tb_hrefs_clirad()
    
    s_html = df.to_html(escape=False)
    display.display(display.HTML(s_html))

In [134]:
script()

lblnew,1,2,3a,3b,3c,4,5,6,7,8,9
clirad,1,2,3,4,5,6,7,8,9,10,11
h2o,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp
co2,,,mls saw trp,mls saw trp,mls saw trp,mls saw trp,mls saw trp,,,,mls saw trp
o3,,,,,,,mls saw trp,,,,mls saw trp
n2o,,,mls saw trp,,,,,,mls saw trp,,
ch4,,,,,,,,,mls saw trp,,
co2+h2o+n2o,,,mls saw trp,,,,,,,,
co2+h2o,,,,mls saw trp,mls saw trp,mls saw trp,,,,,mls saw trp
co2+h2o+o3,,,,,,,mls saw trp,,,,
h2o+o3,,,,,,,mls saw trp,,,,


In [135]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')